# Solución María

### 0. Importaciones

In [7]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('./Data/sql-murder-mystery.db')
cursor_ch = conn.cursor()
sql_query = lambda query: pd.read_sql(query, conn)

### 1. Datos iniciales

In [8]:
query = '''
SELECT * FROM crime_scene_report
WHERE date = 20180115 
AND type = 'murder'
AND city = 'SQL City'
'''

sql_query(query)

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


### 2. Búsqueda de testigos y acceso a sus interrogatorios

In [4]:
query = '''
SELECT 
	person.id, 
	person.name,
	person.address_street_name,
	person.address_number,
	interview.transcript
FROM person
JOIN interview ON person.id = interview.person_id
WHERE address_street_name ='Northwestern Dr' -- Testigo 1: vive en la última casa (MAX()) de la calle Northwestern Dr
AND address_number = (SELECT MAX(address_number) FROM person WHERE address_street_name ='Northwestern Dr')
OR address_street_name = 'Franklin Ave' -- Testigo 2: viven en la calle Franklin Ave y su nombre es 'Annabel', no sabemos su apellido
AND name LIKE 'Annabel%'
'''
sql_query(query)

,id,name,address_street_name,address_number,transcript
0,14887,Morty Schapiro,Northwestern Dr,4919,I heard a gunshot and then saw a man run out. ...
1,16371,Annabel Miller,Franklin Ave,103,"I saw the murder happen, and I recognized the ..."


### 3. Encontrar al asesino

In [9]:
query = '''
SELECT 
	person.id, person.name, person.license_id, 
	drivers_license.plate_number,
	get_fit_now_member.id,
	get_fit_now_check_in.check_in_date,
	get_fit_now_check_in.check_in_time,
	get_fit_now_check_in.check_out_time
FROM person
JOIN drivers_license ON person.license_id = drivers_license.id
JOIN get_fit_now_member ON person.id = get_fit_now_member.person_id
JOIN get_fit_now_check_in ON get_fit_now_member.id = get_fit_now_check_in.membership_id
WHERE plate_number LIKE '%H42W%' -- Tiene una matrícula como la que dice Morty
AND membership_id LIKE '48Z%' -- Tiene suscripción al gimnasio que empieza por 48Z
AND membership_status = 'gold' -- Miembro gold
OR person_id = 16371 -- Coincide con Annabel en el gimnasio
'''

sql_query(query)

,id,name,license_id,plate_number,id,check_in_date,check_in_time,check_out_time
0,67318,Jeremy Bowers,423327,0H42W2,48Z55,20180109,1530,1700
1,16371,Annabel Miller,490173,23AM98,90081,20180109,1600,1700


### 4. Interrogatorio del primer asesino e información sobre el verdadero cerebro

In [10]:
query = '''
SELECT * FROM interview
WHERE person_id = 67318
'''
sql_query(query)

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


### 5. Identificación del verdadero asesino

In [11]:
query = '''
SELECT 
	person.id,
	person.name,
	drivers_license.height,
	drivers_license.hair_color,
	drivers_license.car_make,
	drivers_license.car_model,
	income.annual_income
FROM person
JOIN drivers_license ON person.license_id = drivers_license.id
JOIN income ON person.ssn = income.ssn
JOIN facebook_event_checkin ON person.id = facebook_event_checkin.person_id
WHERE height BETWEEN 65 AND 67 -- Altura entre 5'5" y 5'6"
AND hair_color = 'red' -- Pelirroja
AND car_make = 'Tesla' -- Marca del coche
AND car_model = 'Model S' -- Modelo del coche
AND date BETWEEN 20171201 AND 20171231 -- Asiste 3 veces al SQL Symphony Concert en diciembre 2017
AND event_name = 'SQL Symphony Concert'
GROUP BY person_id
HAVING COUNT(person_id) = 3
'''

sql_query(query)

,id,name,height,hair_color,car_make,car_model,annual_income
0,99716,Miranda Priestly,66,red,Tesla,Model S,310000
